## Examen 1. Minimizar la función dada

In [1]:
import numpy as np
import pandas as pd
import random

## Función Objetivo

In [2]:
def funcion_objetivo(x):
    # Assuming `x` is a numpy array with two elements, x[0] and x[1]
    sum1 = sum(i * np.cos((i + 1) * x[0] + i) for i in range(1, 6))
    sum2 = sum(i * np.cos((i + 1) * x[1] + i) for i in range(1, 6))
    return sum1 * sum2

## Generar Poblaciónes Iniciales

In [3]:
def inicializar_poblacion(pop_size, bounds, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
    population = []
    for _ in range(pop_size):
        individual = np.random.uniform(bounds[0], bounds[1], size=2)  # Two genes for x1 and x2
        population.append(individual)
    return np.array(population)

## Evaluar la poblacion

In [4]:
def evaluacion_aptitud(poblacion):
    n = len(poblacion)
    aptitud = []
    matriz_con_aptitud = pd.DataFrame()
    
    for individuo in poblacion:
        aptitud.append(-funcion_objetivo(individuo))  # Evaluamos la aptitud usando la función objetivo
    
    matriz_con_aptitud["POBLACION"] = [list(ind) for ind in poblacion]  # Guardamos cada individuo
    matriz_con_aptitud["APTITUD"] = aptitud  # Aptitud de cada individuo
    
    return matriz_con_aptitud

# Ejemplo de uso

## Metodos de selección

### Ranking

In [5]:
def rank_seleccion(poblacion):
    n = len(poblacion)  # Número de individuos en la población
    poblacion_hijos = []
    i = 0
    
    # Generamos hijos mediante cruce de pares adyacentes en la población ordenada
    while i < n - 1:
        padre1 = poblacion[i]
        padre2 = poblacion[i + 1]
        
        # Realizamos cruce (un punto o homogéneo)
        hijo_1, hijo_2 = cruza_un_corte(padre1, padre2)  # Puedes cambiar a cruza_homogenea si lo prefieres
        
        poblacion_hijos.append(hijo_1)
        poblacion_hijos.append(hijo_2)
        
        i += 2  # Avanzamos de dos en dos para el cruce de pares
    
    return poblacion_hijos

### Tournament

In [6]:
def tournament(poblacion):
    n = len(poblacion)  # número de individuos en la población
    num_genes = len(poblacion[0])  # número de genes por individuo
    poblacion_hijos = []

    for i in range(n // 2):
        # Tomamos padres en pares (por ejemplo: 1-10, 2-9, etc., si la población está ordenada)
        padre1 = poblacion[i]
        padre2 = poblacion[n - 1 - i]
        
        # Aplicamos cruce homogéneo
        #hijo_1, hijo_2 = cruza_homogenea(padre1, padre2)
        hijo_1, hijo_2 = cruza_un_corte(padre1, padre2)
        
        # Añadimos los hijos generados a la nueva población de hijos
        poblacion_hijos.append(hijo_1)
        poblacion_hijos.append(hijo_2)
    
    return poblacion_hijos


### Aleatorio Monogamico

In [7]:
def aleatorio_monogamico(poblacion):
    n = len(poblacion)  # Número de individuos en la población
    poblacion_disponible = poblacion.copy().tolist()  # Copia de la población para seleccionar y eliminar individuos
    poblacion_hijos = []
    
    # Generamos n/2 pares de padres para obtener una nueva población de hijos
    while len(poblacion_hijos) < n:
        # Selección aleatoria de una pareja de padres
        pareja = random.sample(poblacion_disponible, 2)
        
        # Aplicamos cruce homogéneo entre los padres seleccionados
        hijo_1, hijo_2 = cruza_homogenea(np.array(pareja[0]), np.array(pareja[1]))
        
        # Añadimos los hijos generados a la nueva población de hijos
        poblacion_hijos.append(hijo_1)
        poblacion_hijos.append(hijo_2)
        
        # Eliminamos los padres seleccionados de la población disponible para evitar repetición
        poblacion_disponible.remove(pareja[0])
        poblacion_disponible.remove(pareja[1])
    
    return poblacion_hijos[:n]  # Retornamos exactamente n hijos

## Metodos de Cruza

### Cruza por un corte

In [8]:
def cruza_un_corte(padre1, padre2):
    num_genes = len(padre1)
    punto_corte = num_genes // 2  # Definimos el punto de corte en la mitad

    # Creamos los hijos usando el cruce en el punto definido
    hijo_1 = np.concatenate((padre1[:punto_corte], padre2[punto_corte:]))
    hijo_2 = np.concatenate((padre2[:punto_corte], padre1[punto_corte:]))
    
    return hijo_1, hijo_2

### Cruza Homogenea

In [9]:
def cruza_homogenea(padre1, padre2):
    num_genes = len(padre1)
    hijo_1 = np.zeros(num_genes)
    hijo_2 = np.zeros(num_genes)
    
    for i in range(num_genes):
        # Seleccionamos aleatoriamente si cada gen viene del padre1 o del padre2
        if random.random() < 0.5:
            hijo_1[i] = padre1[i]
            hijo_2[i] = padre2[i]
        else:
            hijo_1[i] = padre2[i]
            hijo_2[i] = padre1[i]
    
    return hijo_1, hijo_2


## Mutación

In [26]:
def mutacion(poblacion, tasa_mutacion=0.1, rango_mutacion=0.5):
    # Asegurarse de que poblacion sea un DataFrame
    if not isinstance(poblacion, pd.DataFrame):
        poblacion = pd.DataFrame(poblacion)
        
    num_individuos = len(poblacion)
    num_genes = len(poblacion['POBLACION'].iloc[0])  # Asumiendo que 'POBLACION' tiene listas
    poblacion_mutada = poblacion.copy()

    print(f"Forma de poblacion_mutada antes de la mutación: {poblacion_mutada.shape}")
    print(f"Columnas de poblacion_mutada: {poblacion_mutada.columns.tolist()}")

    # Calculamos el índice para el peor 10% de la población
    num_mutar = max(1, int(0.1 * num_individuos))

    # Seleccionamos los peores 10%
    indices_peor_10 = range(num_individuos - num_mutar, num_individuos)

    # Aplicamos la mutación solo a los individuos en los índices seleccionados
    for i in indices_peor_10:
        for j in range(num_genes):
            if np.random.rand() < tasa_mutacion:
                mutacion = np.random.uniform(-rango_mutacion, rango_mutacion)
                # Accedemos al valor específico en la lista
                valor_actual = poblacion_mutada['POBLACION'].iloc[i][j]  # Obtener el valor del gen
                print(f"Mutando individuo {i}, gen {j}: {valor_actual} -> ", end="")

                # Mutamos el valor específico
                valor_mutado = valor_actual + mutacion
                poblacion_mutada['POBLACION'].iloc[i][j] = np.clip(valor_mutado, -10, 10)  # Aseguramos el rango

                print(f"{poblacion_mutada['POBLACION'].iloc[i][j]} (mutación: {mutacion})")

    return poblacion_mutada

## Algoritmo Genetico

In [11]:
def run_GA(poblacion, num_generaciones,pop_size ):

    print("Comienza el Algoritmo Genetico")
    #Variables para guardar el historico de cada generacion
    mean_fitness_history = []
    best_fitness_history = []
    snapshots = {}
    
    for generacion in range(num_generaciones):
        #Guardar valores para comparar entre la primera, mitad y ultima generacion
        if generacion == 0:
            snapshots["first"] = poblacion.copy()
        elif generacion == num_generaciones // 2:
            snapshots["middle"] = poblacion.copy()
        elif generacion == num_generaciones - 1:
            snapshots["last"] = poblacion.copy()
    
        #Evaluar a los pobladores
        poblacion_aptitud = evaluacion_aptitud(poblacion)
        matriz_con_aptitud = poblacion_aptitud.sort_values(by="APTITUD", ascending = True).reset_index(drop=True)

        #Obtener la media y la mejor aptitud
        mean_fitness = np.mean(matriz_con_aptitud['APTITUD'])
        best_fitness = np.max(matriz_con_aptitud['APTITUD'])  # Max because we're using negative values for minimization
        
        #Guardar la media y mejor aptitud
        mean_fitness_history.append(mean_fitness)
        best_fitness_history.append(best_fitness)

        poblacion_hijos = rank_seleccion(matriz_con_aptitud["POBLACION"])

        aptitud_hijos = evaluacion_aptitud(poblacion_hijos)

        matriz_mixed = pd.concat([matriz_con_aptitud, aptitud_hijos], ignore_index=True)

        matriz_ordenada = matriz_mixed.sort_values(by="APTITUD", ascending = True).reset_index(drop=True)

        poblacion_mutada = mutacion(matriz_ordenada)
        poblacion_mutada_sorted = poblacion_mutada.sort_values(by="APTITUD", ascending = True).reset_index(drop=True)
        
        poblacion_final = poblacion_mutada_sorted[0:pop_size]

    final_fitness_values = evaluate_population(population)
    best_solution = population[np.argmax(final_fitness_values)]
    best_solution_fitness = -objective_function(best_solution)
        
    return matriz_con_aptitud

In [12]:
pop_size = 20
num_generaciones = 20
mutation_rate = 0.1
elite_size = 2
bounds = [-10, 10]

poblacion1 = inicializar_poblacion(pop_size, bounds, random_state=0)
#poblacion1

In [13]:
matriz_aptitud = evaluacion_aptitud(poblacion1)
matriz_aptitud

,POBLACION,APTITUD
0,"[0.9762700785464951, 4.30378732744839]",13.534334
1,"[2.055267521432878, 0.8976636599379368]",0.516794
2,"[-1.5269040132219054, 2.917882261333123]",-18.517762
3,"[-1.2482557747461502, 7.835460015641594]",2.030385
4,"[9.273255210020587, -2.3311696234844455]",-0.438800
5,"[5.834500761653292, 0.5778983950580887]",-0.049423
6,"[1.3608912218786458, 8.51193276585322]",-7.477312
7,"[-8.579278836042262, -8.257414005969185]",0.550652
8,"[-9.595632051193485, 6.65239691095876]",2.891732
9,"[5.563135018997009, 7.400242964936382]",-8.378415


In [14]:
matriz_con_aptitud = matriz_aptitud.sort_values(by="APTITUD", ascending = True).reset_index(drop=True)
matriz_con_aptitud

,POBLACION,APTITUD
0,"[-0.7704127549413631, 5.610583525729108]",-170.301292
1,"[-0.876993355669029, 1.3686789773729693]",-37.576604
2,"[-7.634511482621336, 2.798420426550477]",-31.531950
3,"[-1.5269040132219054, 2.917882261333123]",-18.517762
4,"[5.563135018997009, 7.400242964936382]",-8.378415
5,"[2.2419144544484286, 2.3386799374951384]",-7.774157
6,"[1.3608912218786458, 8.51193276585322]",-7.477312
7,"[8.874961570292484, 3.636405982069668]",-1.567862
8,"[9.273255210020587, -2.3311696234844455]",-0.438800
9,"[5.834500761653292, 0.5778983950580887]",-0.049423


In [15]:
poblacion_hijos = tournament(matriz_con_aptitud["POBLACION"])
poblacion_hijos

[array([-0.77041275,  5.98317128]),
 array([9.57236684, 5.61058353]),
 array([-0.87699336,  8.89337834]),
 array([-7.13293425,  1.36867898]),
 array([-7.63451148,  4.30378733]),
 array([0.97627008, 2.79842043]),
 array([-1.52690401, -1.7067612 ]),
 array([0.43696644, 2.91788226]),
 array([5.56313502, 2.35270994]),
 array([-9.62420399,  7.40024296]),
 array([2.24191445, 6.65239691]),
 array([-9.59563205,  2.33867994]),
 array([1.36089122, 5.48467379]),
 array([-4.70888776,  8.51193277]),
 array([8.87496157, 7.83546002]),
 array([-1.24825577,  3.63640598]),
 array([ 9.27325521, -8.25741401]),
 array([-8.57927884, -2.33116962]),
 array([5.83450076, 0.89766366]),
 array([2.05526752, 0.5778984 ])]

In [16]:
aptitud_hijos = evaluacion_aptitud(poblacion_hijos)
matriz_mixed = pd.concat([matriz_con_aptitud, aptitud_hijos], ignore_index=True)

matriz_ordenada = matriz_mixed.sort_values(by="APTITUD", ascending = True).reset_index(drop=True)
matriz_ordenada

,POBLACION,APTITUD
0,"[-0.7704127549413631, 5.610583525729108]",-170.301292
1,"[9.572366844655281, 5.610583525729108]",-42.802922
2,"[1.3608912218786458, 5.484673788684333]",-40.987371
3,"[-7.132934251819072, 1.3686789773729693]",-39.208295
4,"[-0.876993355669029, 1.3686789773729693]",-37.576604
5,"[5.563135018997009, 2.352709941517542]",-37.120868
6,"[-1.5269040132219054, -1.7067612001895291]",-36.820631
7,"[-7.634511482621336, 2.798420426550477]",-31.531950
8,"[-1.5269040132219054, 2.917882261333123]",-18.517762
9,"[-1.2482557747461502, 3.636405982069668]",-17.773258


In [17]:
poblacion_mutada = mutacion(matriz_ordenada)
poblacion_mutada_sorted = poblacion_mutada.sort_values(by="APTITUD", ascending = True).reset_index(drop=True)
        
poblacion_final = poblacion_mutada_sorted[0:pop_size]

In [18]:
valor = matriz_ordenada.iloc[37, 0] 
valor

[9.572366844655281, 5.983171284334471]

In [19]:
poblacion_mutada

,POBLACION,APTITUD
0,"[-0.7704127549413631, 5.610583525729108]",-170.301292
1,"[9.572366844655281, 5.610583525729108]",-42.802922
2,"[1.3608912218786458, 5.484673788684333]",-40.987371
3,"[-7.132934251819072, 1.3686789773729693]",-39.208295
4,"[-0.876993355669029, 1.3686789773729693]",-37.576604
5,"[5.563135018997009, 2.352709941517542]",-37.120868
6,"[-1.5269040132219054, -1.7067612001895291]",-36.820631
7,"[-7.634511482621336, 2.798420426550477]",-31.531950
8,"[-1.5269040132219054, 2.917882261333123]",-18.517762
9,"[-1.2482557747461502, 3.636405982069668]",-17.773258


In [20]:
poblacion_final

,POBLACION,APTITUD
0,"[-0.7704127549413631, 5.610583525729108]",-170.301292
1,"[9.572366844655281, 5.610583525729108]",-42.802922
2,"[1.3608912218786458, 5.484673788684333]",-40.987371
3,"[-7.132934251819072, 1.3686789773729693]",-39.208295
4,"[-0.876993355669029, 1.3686789773729693]",-37.576604
5,"[5.563135018997009, 2.352709941517542]",-37.120868
6,"[-1.5269040132219054, -1.7067612001895291]",-36.820631
7,"[-7.634511482621336, 2.798420426550477]",-31.531950
8,"[-1.5269040132219054, 2.917882261333123]",-18.517762
9,"[-1.2482557747461502, 3.636405982069668]",-17.773258


In [27]:
prueba1 = run_GA(poblacion1,num_generaciones,pop_size)

Comienza el Algoritmo Genetico
Forma de poblacion_mutada antes de la mutación: (40, 2)
Columnas de poblacion_mutada: ['POBLACION', 'APTITUD']
Mutando individuo 38, gen 1: 5.983171284334471 -> 6.175643403704491 (mutación: 0.19247211937001985)
Forma de poblacion_mutada antes de la mutación: (40, 2)
Columnas de poblacion_mutada: ['POBLACION', 'APTITUD']
Mutando individuo 36, gen 1: 8.893378340991678 -> 8.969324836547857 (mutación: 0.07594649555617927)
Forma de poblacion_mutada antes de la mutación: (40, 2)
Columnas de poblacion_mutada: ['POBLACION', 'APTITUD']
Mutando individuo 37, gen 0: 9.572366844655281 -> 9.901306873872645 (mutación: 0.3289400292173631)
Mutando individuo 37, gen 1: 5.983171284334471 -> 6.160987821130702 (mutación: 0.1778165367962301)
Forma de poblacion_mutada antes de la mutación: (40, 2)
Columnas de poblacion_mutada: ['POBLACION', 'APTITUD']
Forma de poblacion_mutada antes de la mutación: (40, 2)
Columnas de poblacion_mutada: ['POBLACION', 'APTITUD']
Forma de poblaci

NameError: name 'evaluate_population' is not defined

In [ ]:
prueba1